# L3.  SLAM 

### Define all the imports

In [1]:
%matplotlib inline 
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Change to main repo folder for the imports
_, dir = os.path.split(os.getcwd())
if dir == 'notebooks': 
    os.chdir('..')
    sys.path.append(os.getcwd())
pd.set_option('mode.chained_assignment', None)

In [3]:
from src.graph_SLAM.Graph_SLAM_known_correspondences import GraphSLAM

### Set dataset

In [4]:
# Define dataset to read
dataset = "data/MRCLAM_Dataset1"
robot = 'Robot1' # Robot
start_frame = 2000
end_frame = 5000

### T1. Execute Graph SLAM with known correspondences
Test different datasets and analyze the performance of the algorithm

- What is the main limitation you see with this method?

- Can this method be used to sole the full SLAM problem? and the online SLAM problem?

- Which would be the main differences with respect to the EKF SLAM method?



In [ ]:
# State covariance matrix
R = np.diagflat(np.array([5, 5, 20])) ** 2
# Measurement covariance matrix
Q = np.diagflat(np.array([100.0, 100.0, 1e16])) ** 2
# Iterations
n = 10

graph_slam = GraphSLAM(dataset, robot, start_frame, end_frame, n, R, Q, plot=True)
plt.show()

### T2. Compare with EKF SLAM and FAST SLAM
We are interested in comparing the results obtained with the Graph SLAM with the EKF SLAM and FAST SLAM. Compare the error metrics obtained with the execution of each method.

#### Generate metrics

In [ ]:
#Generate error metrics

#### Represent metrics

In [7]:
# Represent metrics for all experiments

#### Analize metrics

In [8]:
# Analize the results and provide the mean error metrics across the experiments performed.